
# List of Packages to Download

```python
!pip install langchain-chroma
!pip install langchain
!pip install langchain_community
!pip install langchainhub
!pip install -qU langchain-anthropic

```
## Also fix api key


```python

import os

os.environ["ANTHROPIC_API_KEY"] = "your-anthropic-key"
```


______________________________________________________________________________
 

In [1]:
import os
#os.environ["ANTHROPIC_API_KEY"] = "XXXX"
#os.environ["OPENAI_API_KEY"] = "XXXX"

# 1. Get a Data Loader


In [3]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.nba.com/player/2544/lebron-james")
data = loader.load()
data


[Document(page_content='LeBron James | Los Angeles Lakers | NBA.com\n\nNavigation ToggleNBAHomeTickets2023-24 Season ScheduleU.S. Radio BroadcastersLeague Pass ScheduleLivestream ScheduleKey DatesTicketsFeaturedNBA TVNBA Finals: BOS vs. DALLatest Finals NewsScheduleAll SeriesBracketNBA Draft HomeDraft OrderTeamsProspectsDraft FAQHomeTop StoriesDraftFree AgencySummer LeagueTransactionsFeaturesHistoryWriter ArchiveMoreStats HomePlayersTeamsLeadersStats 101Cume StatsLineups ToolMedia Central Game StatsDraftQuick LinksContact UsAtlanticBoston CelticsBrooklyn NetsNew York KnicksPhiladelphia 76ersToronto RaptorsCentralChicago BullsCleveland CavaliersDetroit PistonsIndiana PacersMilwaukee BucksSoutheastAtlanta HawksCharlotte HornetsMiami HeatOrlando MagicWashington WizardsNorthwestDenver NuggetsMinnesota TimberwolvesOklahoma City ThunderPortland Trail BlazersUtah JazzPacificGolden State WarriorsLA ClippersLos Angeles LakersPhoenix SunsSacramento KingsSouthwestDallas MavericksHouston RocketsMe

# 2. Convert data to Vector Database


In [4]:
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

Chroma.from_documents(documents=data, embedding=OpenAIEmbeddings(model="text-embedding-3-large"), persist_directory="./NEWchroma_db")


# 3. Make a RAG pipeline


In [5]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import RetrievalQA
from langchain import hub

In [6]:
from langchain_anthropic import ChatAnthropic

llm = ChatAnthropic(
    model="claude-3-5-sonnet-20240620",
    temperature=0,
    max_tokens=1024,
    timeout=None,
    max_retries=2,
)




In [7]:
prompt = hub.pull("rlm/rag-prompt")
vectorstore = Chroma(persist_directory="./NEWchroma_db", embedding_function=OpenAIEmbeddings(model="text-embedding-3-large"))

In [8]:
qa_chain = RetrievalQA.from_chain_type(
        llm,
        retriever=vectorstore.as_retriever(),
        chain_type_kwargs={"prompt": prompt}
    )

In [13]:
question = "What is the stats from his recent game from april 29th"
result = qa_chain({"query": question })

In [14]:
result

{'query': 'What is the stats from his recent game from april 29th',
 'result': "I don't have specific stats from LeBron James' game on April 29th, 2024 in the context provided. The most recent game information shown is from April 29, 2024, but it only lists the minutes played (44) without other detailed statistics. For the most up-to-date and accurate game stats, it would be best to check official NBA sources or recent news reports."}